In [1]:
import datetime as dt
from utility import (
    read_sql,
    fd_basicinfo,
    nearest_report_date
)
import pandas as pd

In [2]:
# 获取回溯的报告日

today = dt.datetime.today()
date_3yrs_ago = today.replace(year=today.year - 3)
report_dates_begin = nearest_report_date(date_3yrs_ago.strftime("%Y%m%d"))
current_date = today.strftime("%Y%m%d")

In [3]:
# 获取所有目标混合型基金（清算结束日期晚于当前日）

security_ids = read_sql(f"""
select SECURITYID from TQ_FD_TYPECLASS 
WHERE
    ISVALID = 1 AND
    L1CODE = 2 AND
    (ENDDATE>='{report_dates_begin}' or ENDDATE = '19000101') 
ORDER BY SECURITYID;
""")["SECURITYID"].unique().tolist()
sec_id_strs = ",".join(["'" + s + "'" for s in security_ids])

not_liq_sec_ids = fd_basicinfo(security_ids, current_date)["SECURITYID"].unique().tolist()
sec_id_strs = ",".join(["'" + s + "'" for s in not_liq_sec_ids])

In [4]:
# 获取相关组合情况

query = f"""
SELECT SECURITYID, REPORTDATE, EQUITYINVERTO from TQ_FD_ASSETPORTFOLIO
WHERE
    REPORTDATE >= '{report_dates_begin}' AND
    ISVALID = 1 AND
    SECURITYID in ({sec_id_strs})
"""
portfolios = read_sql(query).sort_values("SECURITYID")
agg_port = portfolios.groupby("SECURITYID", as_index=False).mean()
security_ids_has_port = agg_port.SECURITYID.tolist()

In [5]:
# 分类
agg_port.loc[agg_port["EQUITYINVERTO"] >= 70, "混合型（子类）"] = "高权益仓位混合型基金"
agg_port.loc[((agg_port["EQUITYINVERTO"] >= 50) & (agg_port["EQUITYINVERTO"] < 70)), "混合型（子类）"] = "中高权益仓位混合型基金"
agg_port.loc[((agg_port["EQUITYINVERTO"] >= 30) & (agg_port["EQUITYINVERTO"] < 50)), "混合型（子类）"] = "中权益仓位混合型基金"
agg_port.loc[((agg_port["EQUITYINVERTO"] >= 0) & (agg_port["EQUITYINVERTO"] < 30)), "混合型（子类）"] = "低权益仓位混合型基金"

In [7]:
agg_port = pd.merge(agg_port, fd_basicinfo(agg_port.SECURITYID.tolist(), current_date))

In [8]:
agg_port.loc[agg_port["混合型（子类）"] == "低权益仓位混合型基金", :]

,SECURITYID,EQUITYINVERTO,混合型（子类）,FDNAME,SNAMECOMP,FSYMBOL,FDNATURE,INVESTSTYLE
40,1030000079,17.740000,低权益仓位混合型基金,大成财富管理2020生命周期证券投资基金,None,090006,证券投资基金,稳健成长型
54,1030000112,24.380000,低权益仓位混合型基金,国投瑞银融华债券型证券投资基金,None,121001,证券投资基金,收益型
62,1030000124,19.025833,低权益仓位混合型基金,银河收益证券投资基金,None,151002,证券投资基金,稳健成长型
94,1030000180,26.726667,低权益仓位混合型基金,南方宝元债券型基金,南方宝元债券,202101,证券投资基金,稳健成长型
95,1030000185,1.710000,低权益仓位混合型基金,鹏华弘泰灵活配置混合型证券投资基金,鹏华弘泰灵活配置混合,206001,证券投资基金,稳健成长型
...,...,...,...,...,...,...,...,...
2067,1030009757,23.222500,低权益仓位混合型基金,东方价值挖掘灵活配置混合型证券投资基金,东方价值挖掘灵活配置混合,004166,证券投资基金,稳健成长型
2070,1030009810,22.155833,低权益仓位混合型基金,易方达瑞富灵活配置混合型证券投资基金,易方达瑞富混合,001745,证券投资基金,稳健成长型
2071,1030009811,22.155833,低权益仓位混合型基金,易方达瑞富灵活配置混合型证券投资基金,易方达瑞富混合,001746,证券投资基金,稳健成长型
2072,1030009813,21.400833,低权益仓位混合型基金,南方安康混合型证券投资基金,None,004517,证券投资基金,稳健成长型


In [9]:
agg_port.groupby("混合型（子类）")[["FSYMBOL"]].count()

,FSYMBOL
混合型（子类）,
中权益仓位混合型基金,144
中高权益仓位混合型基金,197
低权益仓位混合型基金,508
高权益仓位混合型基金,1205
